### iterable
Any object from which the iter built-in function can obtain an iterator. Objects
implementing an \__iter__ method returning an iterator are iterable. Sequences
are always iterable, as are objects implementing a \__getitem__ method that
accepts 0-based indexes.

In [1]:
#Some people, when confronted with a problem, think "I know, I'll use regular expressions." 
#Now they have two problems.
import re
import reprlib

RE_WORD = re.compile(r'\w+')
RE_WORD.findall('1 2 3')

['1', '2', '3']

In [2]:
class Sentence:
    def __init__(self, text):
        self.text = text
        self.words = RE_WORD.findall(text)
    
    def __getitem__(self, index):
         return self.words[index]
        
    def __len__(self):
        return len(self.words)
    
    def __repr__(self):
        return 'Sentence(%s)' % reprlib.repr(self.text) #for abbreviation

In [3]:
s = Sentence('"the time has come," the Walrus said,')
s

Sentence('"the time ha... Walrus said,')

In [4]:
#whenever python needs to iterate over an object x, it automatically calls iter(x)
#if __iter__ is missing iter() will call __getitem__()
for word in s:
    print(word)

the
time
has
come
the
Walrus
said


In [12]:
list(s)

['the', 'time', 'has', 'come', 'the', 'Walrus', 'said']

In [13]:
set(s)

{'Walrus', 'come', 'has', 'said', 'the', 'time'}

In [14]:
from collections import abc

class GooseSpam:
    def __iter__(self):
        pass


goose_spam_can = GooseSpam()

issubclass(GooseSpam, abc.Iterable), isinstance(goose_spam_can, abc.Iterable)

(True, True)

In [15]:
import random

def d6():
    return random.randint(1, 6)

In [16]:
d6_iter = iter(d6, 1) 
next(d6_iter), type(d6_iter)

(2, callable_iterator)

In [17]:
for roll in d6_iter:
    print(roll)

5
6
3
4
6
2


In [18]:
s = 'ABC'
it = iter(s) 

while True:
    try:
        print(next(it)) 
    except StopIteration: 
        del it 
        break 

A
B
C


### Python’s standard interface for an iterator has two methods:
\__next__:
Returns the next item in the series, raising StopIteration if there are no more.

\__iter__:
Returns self; this allows iterators to be used where an iterable is expected, for
example, in a for loop.

In [19]:
s3 = Sentence('Life of Brian')
it = iter(s3)

In [20]:
next(it), next(it), next(it)

('Life', 'of', 'Brian')

In [21]:
list(it)

[]

In [22]:
list(iter(s3))

['Life', 'of', 'Brian']

In [23]:
class Sentence:
    def __init__(self, text):
        self.text = text
        self.words = RE_WORD.findall(text)
    
    def __repr__(self):
        return f'Sentence({reprlib.repr(self.text)})'

    
    def __iter__(self):
        return SentenceIterator(self.words)
    
class SentenceIterator:
    def __init__(self, words):
        self.words = words
        self.index = 0
        
    def __next__(self):
        try:
            word = self.words[self.index]
        except IndexError:
            raise StopIteration()
        self.index += 1
        return word
    
    def __iter__(self):
        return self

In [24]:
s4 = Sentence('hello good bye hi firend')
it = iter(s4)
next(it), list(it)

('hello', ['good', 'bye', 'hi', 'firend'])

A common cause of errors in building iterables and iterators is to confuse the two. To
be clear: iterables have an \__iter__ method that instantiates a new iterator every
time. Iterators implement a \__next__ method that returns individual items, and an
\__iter__ method that returns self.

To “support multiple traversals,” it must be possible to obtain multiple independent
iterators from the same iterable instance, and each iterator must keep its own internal
state, so a proper implementation of the pattern requires each call to iter(my_itera
ble) to create a new, independent, iterator. That is why we need the SentenceItera
tor class in this example

In [25]:
#just ignore this one
class Sentence:
    def __init__(self, text):
        self.text = text
        self.words = RE_WORD.findall(text)
    
    def __repr__(self):
        return f'Sentence({reprlib.repr(self.text)})'

    def __iter__(self):
        return iter(self.words)

In [26]:
s3 = Sentence('hallo good bye hi')
it = iter(s3)
next(it), list(it)

('hallo', ['good', 'bye', 'hi'])

In [27]:
class Sentence2:
    def __init__(self, text):
        self.text = text
        self.words = RE_WORD.findall(text)
    
    def __repr__(self):
        return f'Sentence({reprlib.repr(self.text)})'

    def __iter__(self):
        for word in self.words:
            yield word       

In [28]:
s = Sentence2('hallo good bye hi')
for w in s:
    print(w)
    
it = iter(s)
next(it), list(it)

hallo
good
bye
hi


('hallo', ['good', 'bye', 'hi'])

In [29]:
#generators implement the iterator interface, so they are also iterable

In [30]:
def gen_123():
    yield 1
    yield 2
    yield 3
    
gen_123, gen_123()

(<function __main__.gen_123()>,
 <generator object gen_123 at 0x00000202B1897A00>)

In [31]:
for i in gen_123():
    print(i)

1
2
3


In [32]:
g = gen_123()
next(g), next(g), next(g)

(1, 2, 3)

In [33]:
#when the generator function returns it raises StopIteration
next(g)

StopIteration: 

In [34]:
#the most lazy version of Sentence 
class Sentence:
    def __init__(self, text):
        self.text = text
        
    def __repr__(self):
        return f'Sentence({reprlib.repr(self.text)})'

    def __iter__(self):
        for match in RE_WORD.finditer(self.text):
            yield match.group()

In [35]:
class Sentence:
    def __init__(self, text):
        self.text = text
        
    def __repr__(self):
        return f'Sentence({reprlib.repr(self.text)})'

    def __iter__(self):
        return (match.group() for match in RE_WORD.finditer(self.text))

In [36]:
s = Sentence('1 2 3 hallo good bye')
it = iter(s)
next(it), list(it)

('1', ['2', '3', 'hallo', 'good', 'bye'])

In [37]:
for i in RE_WORD.finditer('1 2 3'):
    print(i)

<re.Match object; span=(0, 1), match='1'>
<re.Match object; span=(2, 3), match='2'>
<re.Match object; span=(4, 5), match='3'>


In [38]:
def gen_AB():
    print('start')
    yield 'A'
    print('continue')
    yield 'B'
    print('end')

In [39]:
res1 = [3*x for x in gen_AB()]

start
continue
end


In [40]:
res1

['AAA', 'BBB']

In [41]:
res2 = (3*x for x in gen_AB())
res2

<generator object <genexpr> at 0x00000202B24ED700>

In [42]:
for i in res2:
    print('-->', i)

start
--> AAA
continue
--> BBB
end


In [43]:
#generator expression
(2 * x for x in [1,2,3])

<generator object <genexpr> at 0x00000202B24E4C70>

In [44]:
type(8.0)(6)

6.0

In [45]:
def aritprog_gen(begin, step, end=None):
    result = type(begin+step)(begin)
    index = 0
    while (end is None) or result < end:
        yield result
        index += 1
        result = begin + step * index

In [46]:
list(aritprog_gen(1, .2, 3))

[1.0, 1.2, 1.4, 1.6, 1.8, 2.0, 2.2, 2.4000000000000004, 2.6, 2.8]

In [47]:
from decimal import Decimal
list(aritprog_gen(1, Decimal('.2'), 3))

[Decimal('1'),
 Decimal('1.2'),
 Decimal('1.4'),
 Decimal('1.6'),
 Decimal('1.8'),
 Decimal('2.0'),
 Decimal('2.2'),
 Decimal('2.4'),
 Decimal('2.6'),
 Decimal('2.8')]

In [48]:
from fractions import Fraction
list(aritprog_gen(1, Fraction(1,3), 3))

[Fraction(1, 1),
 Fraction(4, 3),
 Fraction(5, 3),
 Fraction(2, 1),
 Fraction(7, 3),
 Fraction(8, 3)]

In [49]:
import itertools

In [50]:
gen = itertools.count(1, .5) #never stops so don't use list here
next(gen), next(gen), next(gen)

(1, 1.5, 2.0)

In [51]:
gen = itertools.takewhile(lambda n: n < 3, itertools.count(1, .5))
list(gen)

[1, 1.5, 2.0, 2.5]

In [52]:
def aritprog(begin, step, end=None):
    first = type(begin + step)(begin)
    ap_gen = itertools.count(first, step)
    if end is None:
        return ap_gen
    return itertools.takewhile(lambda n: n < end, ap_gen)

In [53]:
list(aritprog(0,1,10))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [54]:
from itertools import compress, dropwhile, accumulate, starmap, filterfalse, islice, takewhile

In [55]:
def vowel(c):
    return c.lower() in 'aeiou'

In [56]:
list(filter(vowel, 'Aardvark'))

['A', 'a', 'a']

In [57]:
list(filterfalse(vowel, 'Aardvark'))

['r', 'd', 'v', 'r', 'k']

In [85]:
list(dropwhile(vowel, 'Aardvark'))

['r', 'd', 'v', 'a', 'r', 'k']

In [58]:
list(takewhile(vowel, 'Aardvark'))

['A', 'a']

In [59]:
list(compress('Aardvark', (1, 0, 1, 1, 0, 1)))

['A', 'r', 'd', 'a']

In [60]:
list(islice('Aardvark', 4))

['A', 'a', 'r', 'd']

In [61]:
list(islice('Aardvark', 4, 7))

['v', 'a', 'r']

In [62]:
list(islice('Aardvark', 1, 7, 2))

['a', 'd', 'a']

In [64]:
sample = [5, 4, 2, 8, 7, 6, 3, 0, 9, 1]

In [65]:
list(accumulate(sample))

[5, 9, 11, 19, 26, 32, 35, 35, 44, 45]

In [66]:
list(accumulate(sample, min))

[5, 4, 2, 2, 2, 2, 2, 0, 0, 0]

In [67]:
list(accumulate(sample, max))

[5, 5, 5, 8, 8, 8, 8, 8, 9, 9]

In [69]:
import operator
list(accumulate(sample, operator.mul))

[5, 20, 40, 320, 2240, 13440, 40320, 0, 0, 0]

In [70]:
list(accumulate(range(1,11), operator.mul)) #factorial

[1, 2, 6, 24, 120, 720, 5040, 40320, 362880, 3628800]

In [79]:
list(starmap(operator.mul, enumerate('albatroz', 1)))

['a', 'll', 'bbb', 'aaaa', 'ttttt', 'rrrrrr', 'ooooooo', 'zzzzzzzz']

In [80]:
list(starmap(operator.mul, enumerate('albatroz', 3)))

['aaa',
 'llll',
 'bbbbb',
 'aaaaaa',
 'ttttttt',
 'rrrrrrrr',
 'ooooooooo',
 'zzzzzzzzzz']

In [84]:
list(starmap(lambda a, b: b/a, enumerate(accumulate(sample), 1)))

[5.0,
 4.5,
 3.6666666666666665,
 4.75,
 5.2,
 5.333333333333333,
 5.0,
 4.375,
 4.888888888888889,
 4.5]

In [86]:
from itertools import chain, product, zip_longest

In [87]:
list(chain('ABC', range(3)))

['A', 'B', 'C', 0, 1, 2]

In [96]:
#chain.from_iterable takes each item from the iterable, and chains them in sequence,
#as long as each item is itself iterable
list(chain.from_iterable(enumerate('ABC')))

[0, 'A', 1, 'B', 2, 'C']

In [95]:
list(enumerate('ABC'))

[(0, 'A'), (1, 'B'), (2, 'C')]

In [98]:
list(product(range(3), range(3))) #cartesian product

[(0, 0), (0, 1), (0, 2), (1, 0), (1, 1), (1, 2), (2, 0), (2, 1), (2, 2)]

In [107]:
suits = 'spades diamonds hearts clubs'.split()
suits

['spades', 'diamonds', 'hearts', 'clubs']

In [108]:
list(product(suits, 'AK'))

[('spades', 'A'),
 ('spades', 'K'),
 ('diamonds', 'A'),
 ('diamonds', 'K'),
 ('hearts', 'A'),
 ('hearts', 'K'),
 ('clubs', 'A'),
 ('clubs', 'K')]

In [103]:
[(i, j) for i in range(3) for j in range(3)] #also cartesian product

[(0, 0), (0, 1), (0, 2), (1, 0), (1, 1), (1, 2), (2, 0), (2, 1), (2, 2)]

In [99]:
list(zip_longest('ABC', range(5)))

[('A', 0), ('B', 1), ('C', 2), (None, 3), (None, 4)]

In [100]:
list(zip_longest('ABC', range(5), fillvalue='?'))

[('A', 0), ('B', 1), ('C', 2), ('?', 3), ('?', 4)]

In [109]:
list(product('ABC', repeat=2))

[('A', 'A'),
 ('A', 'B'),
 ('A', 'C'),
 ('B', 'A'),
 ('B', 'B'),
 ('B', 'C'),
 ('C', 'A'),
 ('C', 'B'),
 ('C', 'C')]

In [112]:
list(product(range(2), repeat=3))

[(0, 0, 0),
 (0, 0, 1),
 (0, 1, 0),
 (0, 1, 1),
 (1, 0, 0),
 (1, 0, 1),
 (1, 1, 0),
 (1, 1, 1)]

In [113]:
list(product('AB', range(2), repeat=2))

[('A', 0, 'A', 0),
 ('A', 0, 'A', 1),
 ('A', 0, 'B', 0),
 ('A', 0, 'B', 1),
 ('A', 1, 'A', 0),
 ('A', 1, 'A', 1),
 ('A', 1, 'B', 0),
 ('A', 1, 'B', 1),
 ('B', 0, 'A', 0),
 ('B', 0, 'A', 1),
 ('B', 0, 'B', 0),
 ('B', 0, 'B', 1),
 ('B', 1, 'A', 0),
 ('B', 1, 'A', 1),
 ('B', 1, 'B', 0),
 ('B', 1, 'B', 1)]

In [121]:
from itertools import (combinations, 
                       combinations_with_replacement,
                       cycle,
                       pairwise,
                       permutations,
                       repeat)

In [127]:
from itertools import cycle as c, pairwise as p

In [128]:
cy = cycle('ABC')
list(islice(cy, 7))

['A', 'B', 'C', 'A', 'B', 'C', 'A']

In [129]:
list(pairwise(range(7)))

[(0, 1), (1, 2), (2, 3), (3, 4), (4, 5), (5, 6)]

In [133]:
rp = repeat(4, 5)
list(rp)

[4, 4, 4, 4, 4]

In [134]:
list(map(operator.mul, range(11), repeat(5)))

[0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50]

In [135]:
list(combinations('ABC', 2))

[('A', 'B'), ('A', 'C'), ('B', 'C')]

In [136]:
list(combinations_with_replacement('ABC', 2))

[('A', 'A'), ('A', 'B'), ('A', 'C'), ('B', 'B'), ('B', 'C'), ('C', 'C')]

In [137]:
list(permutations('ABC', 2))

[('A', 'B'), ('A', 'C'), ('B', 'A'), ('B', 'C'), ('C', 'A'), ('C', 'B')]